# Tutorial 2 - Variational quantum eigensolver

This tutorial showcases a simplified variational quantum eigensolver (Peruzzo et al 2014). Using the hybrid principle of openqml, we first train a quantum circuit to minimize the energy expectation for a Hamiltonian

$$ \langle \psi | H | \psi \rangle  = 0.1 \langle \psi_{w} | X | \psi_w \rangle + 0.5 \langle \psi_w | Y | \psi_w \rangle.  $$

Here, $| \psi_w \rangle $ is the state after applying the quantum circuit which depends on trainable weights $w$, and $X$, $Y$ denote the Pauli-X and Pauli-Y operator. 

We then turn things around and use a fixed quantum circuit to prepare $| \psi \rangle $, but train the coefficients of the Hamiltonian to minimize

$$ \langle \psi | H | \psi \rangle  = w_0 \langle \psi | X | \psi \rangle + w_1 \langle \psi | Y | \psi \rangle . $$


## Imports

Alongside the openqml framework, we import the standard gradient descent optimizer.

In [1]:
import openqml as qm
from openqml import numpy as np
from openqml._optimize import GradientDescentOptimizer

14:32:35 WARNING No OpenQML configuration file found.


TODO: In this tutorial we use the projectq simulator. For this to work, you need to install the projectq plugin. DETAILS!


In [2]:
#dev = qm.device('projectq.simulator', wires=2)
dev = qm.device('default.qubit', wires=2)

## Quantum functions

The quantum circuit of the variational eigensolver is an ansatz that defines a manifold of possible quantum states. We use a "hardcoded" initial superposition, together with two rotations and a CNOT gate. 

In [3]:
def ansatz(weights):

    initial_state = np.array([1, 1, 0, 1])/np.sqrt(3)
    qm.QubitStateVector(initial_state, wires=[0, 1])

    qm.RX(weights[0], [0])
    qm.RY(weights[1], [1])
    qm.CNOT([0, 1])

A variational eigensolvers requires us to evaluate expectations of different Pauli operators. In this example, the Hamiltonian is expressed by only two single-qubit Pauli operators, namely the X and Y operator applied to the first qubit. 

Since these operators do not commute, we need two quantum functions, but they can reuse the same device we created. 

*NOTE: If the Pauli observables referred to different qubits, we could use one quantum function and return a tuple of expectations:*

*return (qm.expectation.PauliX(0), qm.expectation.PauliX(1))*

In [4]:
@qm.qfunc(dev)
def circuit_X(weights):
    ansatz(weights)
    return qm.expectation.PauliX(1)


@qm.qfunc(dev)
def circuit_Y(weights):
    ansatz(weights)
    return qm.expectation.PauliY(1)

## Objective

The objective of a VQE, often called a "cost", is simply a linear combination of the expectations, which defines the expectation of the Hamiltonian we are interested in. 

In [5]:
def cost(weights):
    expX = circuit_X(weights)
    expY = circuit_Y(weights)
    return 0.1*expX + 0.5*expY

This cost defines the following landscape:
 <img src="figures/vqe_q_landscape.png" width="450"> 


## Optimization

We use the gradient descent optimizer from Tutorial 1.

In [6]:
weights0 = np.array([0., 0.])
print('Initial weights:', weights0)

o = GradientDescentOptimizer(0.5)
weights = weights0
for iteration in np.arange(1, 21):
    weights = o.step(cost, weights)
    print('Cost after step {}: {}'.format(iteration, cost(weights)))
print('Optimized weights:', weights)

Initial weights: [0 0]
Cost after step 1: -0.04885188791763629
Cost after step 2: -0.1520885316363529
Cost after step 3: -0.23218318322576306
Cost after step 4: -0.2919525276178529
Cost after step 5: -0.3373750061482431
Cost after step 6: -0.3699464440518563
Cost after step 7: -0.3898312213146038
Cost after step 8: -0.39999850098567463
Cost after step 9: -0.40458706512442877
Cost after step 10: -0.4065264820730262
Cost after step 11: -0.4073233352760281
Cost after step 12: -0.4076473963003147
Cost after step 13: -0.40777885323170654
Cost after step 14: -0.40783221689607096
Cost after step 15: -0.40785392418492195
Cost after step 16: -0.4078627781057823
Cost after step 17: -0.40786640037292715
Cost after step 18: -0.40786788710073507
Cost after step 19: -0.4078684993870676
Cost after step 20: -0.4078687524335624
Optimized weights: [1.57008454 2.67634249]


 <img src="figures/vqe_q_landscape_gd.png" width="450"> 

## Optimizing the Hamiltonian coefficients

Instead of optimizing the circuit parameter, we can also use a fixed circuit,

In [10]:
def ansatz():

    initial_state = np.array([1, 1, 0, 1])/np.sqrt(3)
    qm.QubitStateVector(initial_state, wires=[0, 1])

    qm.RX(-0.5, [0])
    qm.RY( 0.5, [1])
    qm.CNOT([0, 1])
    
    
@qm.qfunc(dev)
def circuit_X():
    ansatz()
    return qm.expectation.PauliX(1)


@qm.qfunc(dev)
def circuit_Y():
    ansatz()
    return qm.expectation.PauliY(1)

and make the classical coefficients trainable:

In [8]:
def cost(weights):
    expX = circuit_X()
    expY = circuit_Y()
    return weights[0]*expX + weights[1]*expY

The optimization landscape becomes nearly linear (since smaller coefficients decreas the energy expectation).

In [11]:
weights0 = np.array([0., 0.])
print('Initial weights:', weights0)

o = GradientDescentOptimizer(0.5)
weights = weights0
for iteration in np.arange(1, 21):
    weights = o.step(cost, weights)
    print('Cost after step {}: {}'.format(iteration, cost(weights)))
print('Optimized weights:', weights)

Initial weights: [0. 0.]
Cost after step 1: -0.14149482652500764
Cost after step 2: -0.2829896530500153
Cost after step 3: -0.4244844795750229
Cost after step 4: -0.5659793061000306
Cost after step 5: -0.7074741326250382
Cost after step 6: -0.8489689591500459
Cost after step 7: -0.9904637856750537
Cost after step 8: -1.1319586122000613
Cost after step 9: -1.2734534387250689
Cost after step 10: -1.4149482652500764
Cost after step 11: -1.5564430917750842
Cost after step 12: -1.697937918300092
Cost after step 13: -1.8394327448250993
Cost after step 14: -1.980927571350107
Cost after step 15: -2.122422397875115
Cost after step 16: -2.2639172244001227
Cost after step 17: -2.40541205092513
Cost after step 18: -2.5469068774501378
Cost after step 19: -2.6884017039751456
Cost after step 20: -2.829896530500153
Optimized weights: [-4.25246528 -3.19617026]


 <img src="figures/vqe_c_landscape_gd.png" width="450"> 

## Optimizing classical and quantum parameters

Of course, we can also optimize "classical" and "quantum" weights together.

In [12]:
def ansatz(weights):
    """ Ansatz of the variational circuit."""

    initial_state = np.array([1, 1, 0, 1])/np.sqrt(3)
    qm.QubitStateVector(initial_state, wires=[0, 1])

    qm.RX(weights[0], [0])
    qm.RY(weights[1], [1])
    qm.CNOT([0, 1])


@qm.qfunc(dev)
def circuit_X(weights):
    """Circuit measuring the X operator"""
    ansatz(weights)
    return qm.expectation.PauliX(1)


@qm.qfunc(dev)
def circuit_Y(weights):
    """Circuit measuring the Y operator"""
    ansatz(weights)
    return qm.expectation.PauliY(1)


def cost(weights):
    """Cost (error) function to be minimized."""

    expX = circuit_X(weights[0:2])
    expY = circuit_Y(weights[0:2])

    return weights[2]*expX + weights[3]*expY

weights0 = np.array([0., 0., 0., 0.])
print('Initial weights:', weights0)

o = GradientDescentOptimizer(0.5)
weights = weights0
for iteration in np.arange(1, 21):
    weights = o.step(cost, weights)
    print('Cost after step {}: {}'.format(iteration, cost(weights)))
print('Optimized weights:', weights)

Initial weights: [0. 0. 0. 0.]
Cost after step 1: -0.20071003671471505
Cost after step 2: -0.3559910601990571
Cost after step 3: -0.44385532443929593
Cost after step 4: -0.4245009303474421
Cost after step 5: -0.3434855527790233
Cost after step 6: -0.3495459445408875
Cost after step 7: -0.5480819865254716
Cost after step 8: -0.9117062625485028
Cost after step 9: -1.362570565807977
Cost after step 10: -1.8478475065762556
Cost after step 11: -2.344375985260513
Cost after step 12: -2.8440059819458843
Cost after step 13: -3.344267418902312
Cost after step 14: -3.8445343585156877
Cost after step 15: -4.344705380002273
Cost after step 16: -4.844799254556266
Cost after step 17: -5.344846931898632
Cost after step 18: -5.8448700467900725
Cost after step 19: -6.344880914960131
Cost after step 20: -6.844885916292804
Optimized weights: [1.57008454 2.67634249 3.95493043 5.84558939]


TODO: Visualisation?